In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

# Load Data

In [4]:
sweep5 = pd.read_csv("ncds_sweep5/tab/ncds5cmi.tab",delimiter="\t",low_memory=False)
sweep5.head(10)

,ncdsid,person,n622_5,n5region,n5gor,resp5cmi,resp5wyt,resp5cl,n500518,n500520,...,child,marchild,hqual33,hqual23,verbal,constrct,keyboard,caring,organise,tenure91
0,N10001N,1,2,6,9,1,1,1,0,2,...,1,2,2,2,3.5,3,3,2.66666666666667,3.4,2
1,N10002P,1,1,5,8,1,1,1,0,1,...,2,5,4,4,4,4,4,3.66666666666667,3.8,2
2,N10004R,1,2,1,2,1,1,1,0,5,...,1,2,2,2,4,3.33333333333333,1.5,3.33333333333333,3.4,3
3,N10007U,1,2,6,9,1,1,1,0,1,...,2,6,3,3,3,3,3.5,3,2.8,2
4,N10009W,1,1,5,7,1,1,1,0,1,...,2,7,2,-1,4,4,3,3.66666666666667,2.2,99
5,N10011Q,1,1,7,5,1,1,1,0,1,...,1,3,3,3,3.5,3.33333333333333,4,3.33333333333333,3.4,3
6,N10012R,1,1,6,9,1,1,1,0,3,...,1,1,4,0,3.5,4,1,3.33333333333333,,4
7,N10013S,1,1,3,4,1,1,1,0,1,...,1,1,1,1,3.5,3.33333333333333,4,4,3.6,2
8,N10015U,1,1,1,1,1,0,0,0,1,...,1,1,1,1,,,,,,2
9,N10016V,1,2,4,6,1,1,1,0,2,...,1,2,1,1,3.5,3.33333333333333,3,3.33333333333333,3.4,2


In [5]:
def get_labels(path,flag):

    # open explain file
    explain = open(path, 'r')

    # read the whole file as a single giant string
    explain_text = explain.read()

    # locate variable target string in the whole file
    target_string_var = 'Variable label ='
    target_loc_var = [m.end() for m in re.finditer(target_string_var, explain_text)]
    var_name_list = []
    # locate type target string in the whole file
    target_string_type = 'the SPSS measurement level is'
    target_loc_type = [m.end() for m in re.finditer(target_string_type, explain_text)]
    type_list = ["Categorical" for i in target_loc_var]

    #Find Numeric or Categorical
    for index, loc in enumerate(target_loc_type):
        # get feature character
        char = explain_text[loc+10]
        if char == "S" or char == "s":
            # it is SPSS data type ORDINAL or NOMINAL
            # it is categorical data
            type_list[index] = "Numeric"

            
    #Find variable label
    if flag == 0:
        for loc in target_loc_var:
            start_point = loc+11
            end_point = explain_text.find('\\\n', loc)
            var_name_list.append(explain_text[start_point:end_point])
    elif flag ==1:
        for loc in target_loc_var:
            start_point = loc+8
            end_point = explain_text.find('\par', loc)
            var_name_list.append(explain_text[start_point:end_point])
        
    explain.close()   
    return var_name_list, type_list

In [6]:
# file path of explain file
sweep5_path = 'ncds_sweep5/mrdoc/allissue/ncds5cmi_ukda_data_dictionary.rtf'
#sweep0_3_path = 'ncds_sweep0-3/mrdoc/allissue/ncds0123_ukda_data_dictionary.rtf'
#sweep0_3_labels, sweep0_3_type = get_labels(sweep0_3_path,1)
sweep5_labels, sweep5_type  = get_labels(sweep5_path,0)
print len(sweep5_labels)

3189


# Data Cleaning

In [7]:
sweep5.columns = sweep5_labels
sweep5.drop("ncdsid serial number",axis=1, inplace=True)
sweep5.head(10)

,PERSON NUMBER,Sex of Cohort Member,Standard region at NCDS5,NCDS5 Government Office Region,Responded to Cohort Member Interview,Responded to What Do You Think Questionnaire,Responded to Your Life Since 1974 Questionnaire,Time of Interview (mins),"CMI:2,A1a) Current main economic activity","CMI:2,A1b) WORKING-No. jobs lasting 1/+ mnths since March 1981",...,DV:Does respondent have child(ren)?,"DV:Sex, Partnership and Children",DV:Highest qual gained at age 33,DV:Highest qual gained at age 23,DV:Possession of Verbal Skills,DV:Possession of Construction Skills,DV:Possession of Keyboard Skills,DV:Possession of Caring Skills,DV:Possession of Organisational Skills,DV:Housing tenure in 1991
0,1,2,6,9,1,1,1,0,2,3,...,1,2,2,2,3.5,3,3,2.66666666666667,3.4,2
1,1,1,5,8,1,1,1,0,1,3,...,2,5,4,4,4,4,4,3.66666666666667,3.8,2
2,1,2,1,2,1,1,1,0,5,2,...,1,2,2,2,4,3.33333333333333,1.5,3.33333333333333,3.4,3
3,1,2,6,9,1,1,1,0,1,4,...,2,6,3,3,3,3,3.5,3,2.8,2
4,1,1,5,7,1,1,1,0,1,1,...,2,7,2,-1,4,4,3,3.66666666666667,2.2,99
5,1,1,7,5,1,1,1,0,1,3,...,1,3,3,3,3.5,3.33333333333333,4,3.33333333333333,3.4,3
6,1,1,6,9,1,1,1,0,3,1,...,1,1,4,0,3.5,4,1,3.33333333333333,,4
7,1,1,3,4,1,1,1,0,1,2,...,1,1,1,1,3.5,3.33333333333333,4,4,3.6,2
8,1,1,1,1,1,0,0,0,1,1,...,1,1,1,1,,,,,,2
9,1,2,4,6,1,1,1,0,2,3,...,1,2,1,1,3.5,3.33333333333333,3,3.33333333333333,3.4,2


In [8]:
#replace blank with nan
sweep5.replace(' ',np.nan,inplace=True)

In [9]:
#so many nan values
def check_nan(df, name):
    print len(df[name][df[name].isnull()])
    
for name in sweep5.columns:
    check_nan(sweep5,name)

0
0
0
62
0
0
0
62
100
2256
9118
9471
563
836
563
11337
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
563
499
11469
2252
2189
2125
2265
2518
2580
2976
2832
2368
10308
11378
11366
11391
11379
11379
11369
11449
2111
2133
2131
2238
8625
8830
2236
2272
2234
2143
2273
2376
2305
6854
2151
2104
7074
7135
62
9740
11269
11374
11262
2119
7705
7842
7728
7721
2170
5801
7551
8498
9460
10354
10915
11232
11407
9885
9885
9899
10068
10790
10004
9874
11106
11103
520
3847
4504
4538
4528
4755
5465
5574
5851
5663
4779
11307
4549
10659
11054
10693
10860
10871
10916
704
2030
62
62
62
62
62
62
62
175
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
11469
113
10904
10847
690
9047
62
62
62
62
8495
8536
62
62
62
62
10428
10448
62
62
8530
62
62
62
62
62
62
62
62
62
9540
62
62
10442
62
62
62
62
62
62
62
62
62
10803
129
2387
9142
62
10708
10555

In [10]:
#convert all the predictors to numeric and replace negtive values as nan
def to_numeric(df):
    #get right predictors, delete dataframe in dataframe
    right_predictors = []
    for column in df.columns.values:
        if df[column].shape == (141,):
            right_predictors.append(column)
            
    #process data        
    x = df.copy()[right_predictors]
    
    for index,column in enumerate(x.columns.values):
        temp = pd.to_numeric(x.iloc[:,index],errors='coerce')
        temp[temp < 0] = np.NaN
        x[column] = temp
    
    return x

In [10]:
to_numeric(sweep5[sweep5.columns[:10]]).head()

,PERSON NUMBER,Sex of Cohort Member,Standard region at NCDS5,NCDS5 Government Office Region,Responded to Cohort Member Interview,Responded to What Do You Think Questionnaire,Responded to Your Life Since 1974 Questionnaire,Time of Interview (mins),"CMI:2,A1a) Current main economic activity","CMI:2,A1b) WORKING-No. jobs lasting 1/+ mnths since March 1981"
0,1,2,6.0,9.0,1,1,1,0.0,2.0,3.0
1,1,1,5.0,8.0,1,1,1,0.0,1.0,3.0
2,1,2,1.0,2.0,1,1,1,0.0,5.0,2.0
3,1,2,6.0,9.0,1,1,1,0.0,1.0,4.0
4,1,1,5.0,7.0,1,1,1,0.0,1.0,1.0


In [11]:
response = "CMI:75 F3a) Epilepsy: Ever suffered"

sweep5[response] = pd.to_numeric(sweep5[response],errors='coerce')

In [12]:
sweep5[response].unique()

array([  3.,   1.,  nan,   2.])

In [13]:
#epilepsy patients!
epi = sweep5[sweep5[response] == 1]
print epi.shape
epi.head()

(141, 3188)


,PERSON NUMBER,Sex of Cohort Member,Standard region at NCDS5,NCDS5 Government Office Region,Responded to Cohort Member Interview,Responded to What Do You Think Questionnaire,Responded to Your Life Since 1974 Questionnaire,Time of Interview (mins),"CMI:2,A1a) Current main economic activity","CMI:2,A1b) WORKING-No. jobs lasting 1/+ mnths since March 1981",...,DV:Does respondent have child(ren)?,"DV:Sex, Partnership and Children",DV:Highest qual gained at age 33,DV:Highest qual gained at age 23,DV:Possession of Verbal Skills,DV:Possession of Construction Skills,DV:Possession of Keyboard Skills,DV:Possession of Caring Skills,DV:Possession of Organisational Skills,DV:Housing tenure in 1991
20,1,2,5,8,1,1,1,0,1,1,...,2,6,4,4,4,4,1,4,3.4,2
53,1,2,10,11,1,1,1,0,1,4,...,2,8,4,3,3.5,NaN,4,3.33333333333333,2.8,2
90,1,1,10,11,1,1,1,0,1,1,...,2,7,0,0,3.5,3.33333333333333,2,4,2.6,99
92,1,2,6,9,1,1,1,0,2,1,...,1,2,1,1,3.5,2,2,3.66666666666667,1.2,2
95,1,2,9,10,1,1,1,0,2,3,...,1,2,1,1,4,3,2,3,2,2


In [14]:
epi = to_numeric(epi)
epi.head()

,PERSON NUMBER,Sex of Cohort Member,Standard region at NCDS5,NCDS5 Government Office Region,Responded to Cohort Member Interview,Responded to What Do You Think Questionnaire,Responded to Your Life Since 1974 Questionnaire,Time of Interview (mins),"CMI:2,A1a) Current main economic activity","CMI:2,A1b) WORKING-No. jobs lasting 1/+ mnths since March 1981",...,DV:Does respondent have child(ren)?,"DV:Sex, Partnership and Children",DV:Highest qual gained at age 33,DV:Highest qual gained at age 23,DV:Possession of Verbal Skills,DV:Possession of Construction Skills,DV:Possession of Keyboard Skills,DV:Possession of Caring Skills,DV:Possession of Organisational Skills,DV:Housing tenure in 1991
20,1,2,5,8,1,1,1,0,1,1.0,...,2,6,4.0,4.0,4.0,4.000000,1.0,4.000000,3.4,2
53,1,2,10,11,1,1,1,0,1,4.0,...,2,8,4.0,3.0,3.5,NaN,4.0,3.333333,2.8,2
90,1,1,10,11,1,1,1,0,1,1.0,...,2,7,0.0,0.0,3.5,3.333333,2.0,4.000000,2.6,99
92,1,2,6,9,1,1,1,0,2,1.0,...,1,2,1.0,1.0,3.5,2.000000,2.0,3.666667,1.2,2
95,1,2,9,10,1,1,1,0,2,3.0,...,1,2,1.0,1.0,4.0,3.000000,2.0,3.000000,2.0,2


In [18]:
#if df[column].unique()<15, dummify it
def get_dummy(df):
    
    #copy dataframes
    data = df.copy()
    
    #Create a new data frame 
    x = pd.DataFrame({}) 
    predictor_list = []

    # Iterate over variables
    for i,column in enumerate(data.columns.values):
        #delete some errors

        if (len(data[column].unique()) < 15):
            #dummify categorical predictors
            encoding = pd.get_dummies(data[column])

            #get dummified predictors column names and store them
            temp = []
            names = encoding.columns.values
            for i in range(len(names)):
                temp.append(column + "=" + str(names[i]))


            # append expanded attribute to data frame
            encoding.columns = temp
            x = pd.concat([x, encoding], axis=1)          
            
#     #if predictors contain info related to epilepsy, delete it
#     for name in x.columns.values:
#         if "epilep" in name and name != "CMI:75 F3a) Epilepsy: Ever suffered":
#             x.drop(name, axis=1, inplace=True)
    
    predictor_list = x.columns.values 

    print x.shape
    print "number of variables: ",len(predictor_list)
    return x

In [ ]:
sweep5_epi = get_dummy(sweep5)
print sweep5_epi.shape
sweep5_epi.head()

# Do Analysis

In [22]:
def compare(name):
    print pd.to_numeric(sweep5[name],errors='coerce').value_counts()/pd.to_numeric(sweep5[name],errors='coerce').value_counts().sum()
    print epi[name].value_counts()/epi[name].value_counts().sum()
    print epi[name].value_counts()

In [24]:
compare("CMI:91 F65a) Car, van: Holds licence")

1.0    0.993122
2.0    0.006878
Name: CMI:91 F65a) Car, van: Holds licence, dtype: float64
1.0    0.986486
2.0    0.013514
Name: CMI:91 F65a) Car, van: Holds licence, dtype: float64
1.0    73
2.0     1
Name: CMI:91 F65a) Car, van: Holds licence, dtype: int64


In [109]:
compare("CML:16,9/1 Unemployed & seeking work since school")

1.0    0.353486
7.0    0.298524
4.0    0.197557
3.0    0.103308
6.0    0.023003
5.0    0.014453
2.0    0.009669
Name: CML:16,9/1 Unemployed & seeking work since school, dtype: float64
1.0    0.453125
7.0    0.218750
4.0    0.156250
5.0    0.132812
3.0    0.039062
Name: CML:16,9/1 Unemployed & seeking work since school, dtype: float64
1.0    58
7.0    28
4.0    20
5.0    17
3.0     5
Name: CML:16,9/1 Unemployed & seeking work since school, dtype: int64


In [ ]:
#add sum info
count = epi.sum(axis = 0)
epi.loc[-1] = count # adding a row
epi.index = epi.index + 1  # shifting index
epi = epi.sort_index()
print epi.shape
epi.head()

In [ ]:
#sort the columns by sum
order = epi.sum(axis = 0).argsort()
sorted_epi = epi[epi.columns.values[order]]
sorted_epi.head()

In [ ]:
#print top
number = sorted_epi.iloc[0,:]
#last = sorted_data.columns.values[:30]
print number[-50:]

# Load Data

In [24]:
df = pd.read_csv("clean_data2.csv",delimiter=",",low_memory=False)
print df.shape
df.head(10)

(11326, 1450)


,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,3.0,NaN,1.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,3.0,3.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,12.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,1.0,0.0,1.0,1.0,8.0,NaN,6.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
6,3.0,NaN,0.0,1.0,1.0,4.0,6.0,3.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
7,3.0,NaN,0.0,1.0,1.0,6.0,8.0,4.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8,3.0,NaN,2.0,2.0,1.0,2.0,4.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,NaN,1.0,3.0,1.0,4.0,6.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


### Get epilepsy patients data

In [25]:
response = "CMI:75 F3a) Epilepsy: Ever suffered"
epi = df[df[response] == 1]
print epi.shape
epi.head()

(141, 1450)


,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
20,1.0,NaN,3.0,2.0,1.0,4.0,6.0,2.0,2.0,2.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1.0,NaN,2.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
90,1.0,NaN,0.0,1.0,1.0,3.0,5.0,3.0,2.0,2.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,1.0,NaN,1.0,3.0,1.0,3.0,5.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
95,1.0,NaN,0.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


### Dummify and only look at categorical values

In [27]:
data = get_dummy(epi)

(141, 4958)
number of variables:  4958


In [28]:
data.head()

,CMI:75 F3a) Epilepsy: Ever suffered=1.0,1P Child 8 years-old at NCDS1 Parental=1.0,1P Family moves since child's birth=0.0,1P Family moves since child's birth=1.0,1P Family moves since child's birth=2.0,1P Family moves since child's birth=3.0,1P Family moves since child's birth=4.0,1P Family moves since child's birth=5.0,1P Family moves since child's birth=6.0,1P Family moves since child's birth=9.0,...,CMI:95 F75a) CHECK: F64 - Has driving licence=2.0,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74)=1.0,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74)=2.0,CMI:95 F75c) Additional traffic offence conviction #1=2.0,CMI:95 F75c) Additional traffic offence conviction #1=3.0,CMI:95 F75c) Additional traffic offence conviction #1=5.0,CMI:95 F75c) Additional traffic offence conviction #2=3.0,CMI:95 F75c) Additional traffic offence conviction #2=4.0,CMI:95 F75c) Additional traffic offence conviction #2=5.0,CMI:95 F75c) Additional traffic offence conviction #3=5.0
20,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
order = data.sum(axis = 0).argsort()
sort = data.columns.values[count.argsort()]

CMI:75 F3a) Epilepsy: Ever suffered=1.0                                  141.0
1P Child 8 years-old at NCDS1 Parental=1.0                                 2.0
1P Family moves since child's birth=0.0                                   42.0
1P Family moves since child's birth=1.0                                   42.0
1P Family moves since child's birth=2.0                                   13.0
1P Family moves since child's birth=3.0                                    6.0
1P Family moves since child's birth=4.0                                    1.0
1P Family moves since child's birth=5.0                                    4.0
1P Family moves since child's birth=6.0                                    2.0
1P Family moves since child's birth=9.0                                    1.0
1P Moves out of local area=1.0                                            42.0
1P Moves out of local area=2.0                                            24.0
1P Moves out of local area=3.0                      

In [46]:
data.loc[-1] = count # adding a row
data.index = data.index + 1  # shifting index
data = data.sort_index()
print data.shape
data.head()

(142, 4958)


,CMI:75 F3a) Epilepsy: Ever suffered=1.0,1P Child 8 years-old at NCDS1 Parental=1.0,1P Family moves since child's birth=0.0,1P Family moves since child's birth=1.0,1P Family moves since child's birth=2.0,1P Family moves since child's birth=3.0,1P Family moves since child's birth=4.0,1P Family moves since child's birth=5.0,1P Family moves since child's birth=6.0,1P Family moves since child's birth=9.0,...,CMI:95 F75a) CHECK: F64 - Has driving licence=2.0,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74)=1.0,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74)=2.0,CMI:95 F75c) Additional traffic offence conviction #1=2.0,CMI:95 F75c) Additional traffic offence conviction #1=3.0,CMI:95 F75c) Additional traffic offence conviction #1=5.0,CMI:95 F75c) Additional traffic offence conviction #2=3.0,CMI:95 F75c) Additional traffic offence conviction #2=4.0,CMI:95 F75c) Additional traffic offence conviction #2=5.0,CMI:95 F75c) Additional traffic offence conviction #3=5.0
0,141.0,2.0,42.0,42.0,13.0,6.0,1.0,4.0,2.0,1.0,...,70.0,14.0,57.0,5.0,6.0,3.0,1.0,1.0,2.0,1.0
21,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
sorted_data = data[order]
sorted_data.head()

,CMI:95 F75c) Additional traffic offence conviction #3=5.0,3P Hopes chlds 1st job - crtnty of answe=4.0,3P Hopes chlds 1st job - crtnty of answe=3.0,3P Why wish child left sch at 15-MC 3:3=6.0,3P Why wish child left sch at 15-MC 2:3=7.0,3P Why wish child left sch at 15-MC 2:3=6.0,3P Why wish chld left sch at 15-MC 1:3=7.0,3P Why wish chld left sch at 15-MC 1:3=2.0,3P How often child goes out in evening=5.0,3P No. of cigarettes dad smokes per day=2.0,...,CMI:75 F3a) Diabetes: Ever suffered=3.0,CMI:76 F8a) INTERVIEWER: CM can/cannotwalk=1.0,CMI:88 F50d) No. glasses martini/vermouth/sherry/ etc in ast 7 days=0.0,CMI:91 F65b) HGV: Provisional/full licence=0.0,CMI:94 F74a) ROAD TRAFFIC ACCIDENT 2: Incident No. (F36a)=0.0,CMI:89 F58a) Sort of bread usually eaten #8=0.0,CMI:89 F58a) Sort of bread usually eaten #7=0.0,CMI:94 F74a) ROAD TRAFFIC ACCIDENT 3: Incident No. (F36a)=0.0,CMI:94 F74a) ROAD TRAFFIC ACCIDENT 4: Incident No. (F36a)=0.0,CMI:75 F3a) Epilepsy: Ever suffered=1.0
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,136.0,138.0,138.0,138.0,140.0,140.0,140.0,141.0,141.0,141.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Most important factors

In [68]:
#top = sorted_data.columns.values[-100:]
number = sorted_data.iloc[0,:]
#last = sorted_data.columns.values[:30]
print number[-400:-350]
#print "top", top
#print "last", last 

1P Awkward,clumsy-when climbing stairs=3.0                               104.0
3P Has the child ever worn a hearng aid=3.0                              104.0
CMI:76 F4a) Any sneezing/runny nose/blocked nose when no cold/flu=2.0    104.0
1M Skull deformity=3.0                                                   105.0
2P Ever seen spec-asthma,whzy bronchitis=1.0                             105.0
2P Child destroys own,others things=1.0                                  105.0
CMI:77 F13 CHECK F9 & F12 - Wheezing/whistling=2.0                       105.0
CMI:76 F9 Ever had wheezing/whistling in the chest=2.0                   105.0
1P Ever hay fever or sneezing attacks=3.0                                105.0
1P Fam contact-Nat Cncl Unmarried Mums=3.0                               105.0
1P Fam contact-Any hndcapped kids assoc=3.0                              105.0
1P Relationship person acting as father=2.0                              105.0
1M Abnormal chest shape=3.0                         

### Filter Factors

In [58]:
features = []

In [61]:
features.append("CMI:91 F65b) HGV: Provisional/full licence")

### Plot and visualize

In [19]:
#a function to deliver comparation
def compare(name):
    #ratio and count among whole population
    print pd.to_numeric(df[name],errors='coerce').value_counts()/pd.to_numeric(df[name],errors='coerce').value_counts().sum()
    print df[name].value_counts()
    
    #ratio and count among epilepsy patients
    print epi[name].value_counts()/epi[name].value_counts().sum()
    print epi[name].value_counts()

### Compare

In [15]:
df = pd.read_csv("clean_data2.csv",delimiter=",",low_memory= False)

In [16]:
df.head()

,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,3.0,NaN,1.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,3.0,3.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,12.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
response = "CMI:75 F3a) Epilepsy: Ever suffered"
epi = df[df[response] == 1]
print epi.shape
epi.head()

(141, 1450)


,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
20,1.0,NaN,3.0,2.0,1.0,4.0,6.0,2.0,2.0,2.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1.0,NaN,2.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
90,1.0,NaN,0.0,1.0,1.0,3.0,5.0,3.0,2.0,2.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,1.0,NaN,1.0,3.0,1.0,3.0,5.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
95,1.0,NaN,0.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
compare("CMI:75 F3a) Persistent mouth/gum trouble: Ever suffered")

3.0    0.943360
1.0    0.052399
2.0    0.004241
Name: CMI:75 F3a) Persistent mouth/gum trouble: Ever suffered, dtype: float64
3.0    10676
1.0      593
2.0       48
Name: CMI:75 F3a) Persistent mouth/gum trouble: Ever suffered, dtype: int64
3.0    0.814286
1.0    0.178571
2.0    0.007143
Name: CMI:75 F3a) Persistent mouth/gum trouble: Ever suffered, dtype: float64
3.0    114
1.0     25
2.0      1
Name: CMI:75 F3a) Persistent mouth/gum trouble: Ever suffered, dtype: int64


In [21]:
def stat(name):
    
    #ratio and count among whole population
    whole = pd.to_numeric(df[name],errors='coerce').value_counts()/pd.to_numeric(df[name],errors='coerce').value_counts().sum()
    whole.sort_index(inplace = True)
    print whole
    
    #ratio and count among epilepsy patients
    e = epi[name].value_counts()/epi[name].value_counts().sum()
    e.sort_index(inplace = True)
    print e
    
    plot(name,)
    


In [22]:
def plot(title, ticks, a, b):

    # create plot
    fig, ax = plt.subplots()
    index = np.arange(len(a))
    bar_width = 0.15/4 * len(a)
    opacity = 0.8

    rects1 = plt.bar(index, a, bar_width,
                     alpha=opacity,
                     color='grey',
                     label='Population',align='center')

    rects2 = plt.bar(index + bar_width, b, bar_width,
                     alpha=opacity,
                     color='red',
                     label='Epilepsy',align='center')

    #plt.xlabel('Person')
    plt.ylabel('Percentage')
    plt.title(title)
    plt.xticks(index + bar_width, ticks)
    plt.legend(loc = "best")

    plt.tight_layout()
    plt.savefig(title +".png")
    plt.show()
    